##2. b Multi-Layer Perceptron Classifier

In [9]:
from pylab import *
%matplotlib inline
from sklearn.cross_validation import KFold
from sklearn.preprocessing import *
from __future__ import division
import random
import math
t=np.seterr(divide='ignore', invalid='ignore')

###Activation Functions

In [10]:
def sigmoid(x):
    return 1/(1.0 + np.exp(-1*x))
def softmax(x, X):
    softMax = np.exp(x)/np.sum(np.exp(X))
    return softMax

###Neural Network Implementation

In [25]:
class NeuralNetwork:
    def __init__(self, setup):
        
        #Setting unit counts
        self.nX = setup['Input Units'] + 1 
        self.nZ = setup['Hidden Units'] + 1
        self.nY = setup['Output Units']

        #Units
        self.x = np.zeros(self.nX)
        self.x[0] = 1
        
        self.z = np.zeros(self.nZ)
        self.z[0] = 1
        
        self.y = np.zeros(self.nY)
        self.target = setup['Targets']
        
        # Weights
        self.w = np.zeros((self.nX, self.nZ-1))
        
        for i in range(self.nX):
            for j in range(self.nZ-1):
                self.w[i][j] = random.uniform(-0.05, 0.05)
        
        self.v = np.zeros((self.nZ, self.nY))
        
        for i in range(self.nZ):
            for j in range(self.nY):
                self.v[i][j] = random.uniform(-0.05, 0.05)
        
        # Delta values from previous iteration
        self.dw = np.zeros((self.nX, self.nZ-1))
        self.dv = np.zeros((self.nZ, self.nY))
        

    def update(self, inputs):
        # input activations
        for i in range(1,self.nX):
            self.x[i] = inputs[i-1]
        
        # hidden activations
        for j in range(1,self.nZ):
            sum = 0.0
            for i in range(self.nX):
                sum += self.x[i] * self.w[i][j-1]
            self.z[j] = sigmoid(sum)

        # output activations
        scores = np.zeros(self.nY)
        for k in range(self.nY):
            score = 0.0
            for j in range(self.nZ):
                score += self.z[j] * self.v[j][k]
            scores[k] = score
        for k in range(self.nY):
            self.y[k] = softmax(scores[k], scores)
        
        return self.y

    def indicator(self, target):
        return [1 if self.target[j] == target else 0 for j in range(self.nY)]
        
    
    def backPropagate(self, target, alpha, momentum):
        
        #Output Units for the target
        targets = self.indicator(target)
        
        # Calculate delta V
        v_deltas = np.zeros((self.nZ, self.nY))
        error = np.zeros(self.nY)
        for j in range(self.nZ):
            for k in range(self.nY):
                error[k] = self.y[k] - targets[k]
                v_deltas[j][k] = error[k] * self.z[j]

        # Update V
        for j in range(self.nZ):
            for k in range(self.nY):
                self.v[j][k] = self.v[j][k] - (alpha * v_deltas[j][k] + momentum * self.dv[j][k])
                self.dv[j][k] = v_deltas[j][k] 
        
        # Calculate delta W
        w_deltas = np.zeros((self.nX, self.nZ-1))
        for i in range(self.nX):
            sumK = 0.0
            for j in range(1,self.nZ):
                for k in range(self.nY):
                    sumK += error[k] * self.v[j][k] * self.z[j] * (1.0 - self.z[j])
                w_deltas[i][j-1] = sumK * self.x[i]

        # Update W
        for i in range(self.nX):
            for j in range(self.nZ-1):
                self.w[i][j] = self.w[i][j] - (alpha * w_deltas[i][j] + momentum * self.dw[i][j])
                self.dw[i][j] = w_deltas[i][j]

        
        # calculate error
        err = 0.0
        for k in range(len(targets)):
            err += 0.5 * (error[k]**2)
        
        return err

    def classify(self, y):
        return self.target[np.argmax(y)]
    
    def getConfusionMatrix(self, actual, prediction):
        m = len(actual)
        cm = np.zeros((self.nY, self.nY))
        for i in range(self.nY):
            actual_i = filter(lambda x: actual[x] == self.target[i], range(m))
            for j in range(self.nY):
                predicted_j = filter(lambda x: prediction[x] == self.target[j], range(m))
                cm[i,j] = len(np.intersect1d(actual_i, predicted_j))
        return cm
    
    def test(self, data):
        error = 0
        m = len(data)
        estimate = np.zeros(m)

        for i in range(m):
            estimate[i] = self.classify(self.update(data[i][:-1]))
            
            if estimate[i] != data[i][-1]:
                error+=1
        
        #print "Error: ", error
        return self.getConfusionMatrix(estimate, data[:,-1])
        
    def weights(self):
        print 'W:'
        for i in range(self.nX):
            print self.w[i]
        print
        print 'V:'
        for j in range(self.nZ):
            print self.v[j]

    def train(self, data, iterations, alpha, momentum):
        for i in range(iterations):
            error = 0.0
            
            for x in data:
                input = x[:-1]
                target = x[-1]
                #print Y
                self.update(input)
                error = error + self.backPropagate(target,alpha, momentum)
            
            #Report Status
            #if i % 100 == 0:
                #print('error %-.8f' % error)
                #print 'X', self.x
                #print 'Z', self.z
                #print 'Y_hat', self.y
                #self.weights()
            
            #Objective Error
            if error <= 0.0001:
                #print('error %-.8f' % error)
                break

###Normalize the given dataset

In [ ]:
def normalize(data):
    
    [nrows, ncols] = data.shape

    col_mean = np.zeros(ncols-1).reshape(ncols-1, 1)
    col_sd = np.zeros(ncols-1).reshape(ncols-1, 1)
    normalized_data = data.astype(float64)
    
    for i in range(ncols-1):
        col_mean[i] = np.mean(data[:,i])
        col_sd = np.std(data[:,i])
        normalized_data[:,i] = [(data[:,i][j] - col_mean[i])/col_sd for j in range(nrows)]
    return normalized_data

###Calculate Performance Metrics

In [13]:
def getAccuracy(matrix):
    return np.trace(matrix)/np.sum(matrix)

def getPrecision(matrix):
    k = len(matrix)
    precision = np.zeros(k)
    for i in range(k):
        actual_positives = np.sum(matrix[i])
        precision[i] = matrix[i,i]/actual_positives
    return precision

def getRecall(matrix):
    k = len(matrix)
    recall = np.zeros(k)
    for i in range(k):
        predicted_positives = np.sum(matrix[:,i])
        recall[i] = matrix[i,i]/predicted_positives
    return recall

def getFMeasure(precision, recall):
    k = len(precision)
    fm = np.zeros(k)
    for i in range(k):
        fm[i] = 2* precision[i]*recall[i]/(precision[i]+recall[i])
    return fm

###SKLearn Implementation MLPClassifier

In [70]:
from sklearn.neural_network import *
#Dev Code Implementation
def inbuilt(train_data, test_data, config):
    (m, n) = train_data.shape
    X_train = train_data[:,:n-1].reshape((m,n-1))
    Y_train = train_data[:,n-1].reshape((m,1))
    (m, n) = test_data.shape
    X_test = test_data[:,:n-1].reshape((m,n-1))
    Y_test = test_data[:,n-1].reshape((m,1))
    
    mlp = MLPClassifier(hidden_layer_sizes=(config['Hidden Units'],), max_iter=10, algorithm='sgd', 
                            momentum=config['Momentum'], learning_rate='constant', learning_rate_init=config['Learning Rate'])
        
    mlp.fit(X_train, Y_train)
        
    print("Training set score: %f" % mlp.score(X_train, Y_train))
    print("Test set score: %f" % mlp.score(X_test, Y_test))
    return

###Run the algorithm

In [71]:
def run(config, data):
    
    targets = np.unique(data[:,-1])
    #print "Targets", targets
    K = config['K-Fold']
    CV_idx = KFold(len(data), n_folds=K)
    
    
    config['Input Units'] = len(data[0]) - 1
    config['Targets'] = targets
    config['Output Units'] = len(targets)
    
    
    cm = np.zeros((len(targets),len(targets)))
    
    for train_idx, test_idx in CV_idx:
        # Creating MLP
        MLP = NeuralNetwork(config)
        
        # Train
        #print "\n***Training Begins***"
        MLP.train(data[train_idx], config['Iterations'], config['Learning Rate'], config['Momentum'])
    
        # Test
        #print "\n***Testing Begins***"
        cm = np.add(cm, MLP.test(data[test_idx]))
        
        #In-built Python Library
        #inbuilt(data[train_idx],data[test_idx],config)
    
    precision = getPrecision(cm)
    accuracy = getAccuracy(cm)
    recall = getRecall(cm)
    FMeasure = getFMeasure(precision, recall)
    
    print "Confusion Matrix"
    print cm
    print "Accuracy\t:\t", accuracy
    print "Precision\t:\t", precision
    print "Recall\t\t:\t", recall
    print "F-Measure\t:\t", FMeasure

###Get the input dataset

In [62]:
def getData(dataset):
    if dataset=='Iris':
        from sklearn import datasets
        data = datasets.load_iris() 
    
        X = data['data']
        Y = data['target']
        Y = Y.reshape(len(data['target']), 1)
        data = np.hstack((X, Y))
        data = normalize(data)
    elif dataset=='mnist':
        from sklearn.datasets import fetch_mldata
        data = fetch_mldata('mnist-original', data_home='C:\\Users\\admin\\Anaconda\\CS584')
        randIdx = np.zeros(1000).astype(int)
        for i in range(1000):
            randIdx[i] = random.randint(0, 69999)
            
        Y = data['target']
        X = data['data']/255.
        Y = Y.reshape(len(Y), 1)
        data = np.hstack((X, Y))
        data = data[randIdx]
    return data

###Main function

In [72]:
if __name__ == '__main__':
    
    config = {
        
    'Hidden Units'  : 5,
    'Learning Rate' : 0.2,
    'Momentum'      : 0.7,
    'Iterations'    : 100,
    'K-Fold'        : 10
        
    }
    
    data = getData('Iris')
    run(config, data)

Confusion Matrix
[[ 50.   1.   0.]
 [  0.  45.   2.]
 [  0.   4.  48.]]
Accuracy	:	0.953333333333
Precision	:	[ 0.98039216  0.95744681  0.92307692]
Recall		:	[ 1.    0.9   0.96]
F-Measure	:	[ 0.99009901  0.92783505  0.94117647]
